# 10. Word Embeddings

## 10.1. Kapitelübersicht <a class="anchor" id="10-1"/>

Bis zu diesem Kapitel hatten wir immer eine Bag-of-Words Repräsentation für unsere Daten benutzt. In Kapitel 4 hatten wir diese durch eine Tf-Idf-Gewichtung erweitert. In diesem Kapitel werden wir ... TODO

<b>Abschnittsübersicht</b><br>

[10.1. Kapitelübersicht](#10-1)<br>


Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- TODO

## 10.2. Grenzen des Bag-of-Words Modell <a class="anchor" id="10-2"/>

Das Bag-of-Words Modell ist sehr einfach zu verstehen und simpel zu implementieren. Es ist flexibel und kann z.B. mit der tf-idf-Gewichtung erweitert werden. Jedoch ist das Bag-of-Words Modell auch limitiert: Eines der größten Schwächen ist die Missachtung der semantischen Beziehung von Wörtern. Synonyme werden ignoriert. Weiterhin hat das Bag-of-Words Modell das Problem, dass je größer das Vokabular wird, desto größer können einzelne Wortvektoren werden. Diese sind sparse, da sie fast nur aus Nullen bestehen. TODO MEHR

MACHINE LEARNING MASTERY

Das Bag-of-Word-Modell ist sehr einfach zu verstehen und zu implementieren und bietet viel Flexibilität bei der Anpassung Ihrer spezifischen Textdaten.

Es wurde mit großem Erfolg bei Vorhersageproblemen wie Sprachmodellierung und Dokumentationsklassifizierung eingesetzt.

Dennoch weist es einige Mängel auf, wie z.

    Wortschatz: Der Wortschatz erfordert eine sorgfältige Gestaltung, insbesondere um die Größe zu verwalten, was sich auf die Sparsamkeit der Dokumentendarstellungen auswirkt.
    Sparsity: Sparse Repräsentationen sind sowohl aus rechnerischen Gründen (Raum- und Zeitkomplexität) als auch aus Informationsgründen schwieriger zu modellieren, wenn die Modelle die Herausforderung haben, so wenig Informationen in einem so großen Repräsentationsraum zu nutzen.

## 10.3. Word Embeddings

<b>Word Embedding</b> (deutsch: Worteinbettung) ist die Sammelbezeichnung für eine Reihe von Sprachmodellierungstechniken. Sie bieten eine andere Form der Wortrepräsentation als das Bag-of-Words-Modell, bei der Wörter mit einer ähnlichen Bedeutung ähnlich dargestellt werden, wobei der <b>Kontext</b> der Wörter berücksichtigt wird. Word Embeddings repräsentieren Wörter als Vektoren in einem multidimensionalen semantischen Raum. Diesen Raum kann man sich etwa folgendermaßen vorstellen:<br><br>

<img src="https://miro.medium.com/max/1291/1*Fat62b1ZITOFMPXTcHNkLw.jpeg" alt="Word Embedding Space" width="600px;"/><br>

In diesem Raum werden Wörter, die ähnlich zueinander sind, näher beieinander platziert. 

TODO MEHR

## 10.4. Nutzung der Word Embeddings

TODO erklären:
- word2vec
- Glove
- fasttext

#### Doc2Vec

http://linanqiu.github.io/2015/10/07/word2vec-sentiment/

In [26]:
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec, Word2Vec, FastText
import pandas as pd

In [51]:
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus.csv", index_col=0)

In [52]:
#Text in Lowercase
#corpus["text"] = corpus["text"].str.lower()

In [55]:
corpus.head()

,category,length,text
id,,,
0,Fernsehserie nach Staat,107,"Tāmar und Schawqīya (arabisch تامر وشوقية, DMG..."
1,Fernsehserie nach Staat,518,Achtung: Streng geheim! (Originaltitel: Missio...
2,Fernsehserie nach Staat,877,Alien Surfgirls ist eine australische Jugendse...
3,Fernsehserie nach Staat,1199,All Saints ist eine australische Fernsehserie ...
4,Fernsehserie nach Staat,142,Alle lieben Diggy ist eine australische Zeiche...


In [56]:
from nltk import sent_tokenize

text = []

for article in corpus["text"]:
    text.append(sent_tokenize(article))

In [57]:
model2 = FastText(text, size=100, window=5, min_count=5, workers=6, sg=1)

In [62]:
model2.wv.most_similar("jahrhundert", topn=10)

[('Chr.).', 0.9991431832313538),
 ('Jahrtausends v. Chr.', 0.9991098046302795),
 ('Chr.)', 0.9991024732589722),
 ('Jahrhunderts v. Chr.', 0.9990993738174438),
 ('Jahrhundert.', 0.9990963935852051),
 ('v. Chr.', 0.9990937113761902),
 ('Jahrhundert zurück.', 0.9990932941436768),
 ('Jahrtausend v. Chr.', 0.9990925192832947),
 ('Chr.', 0.9990907907485962),
 ('Jahrhundert v. Chr.', 0.9990905523300171)]

In [28]:
model = Word2Vec(text, min_count=1, sg=1)

In [29]:
model.save('tutorialdata/embeddings/smallwikicorpus_v2_word2vec.bin') 

In [30]:
model = Word2Vec.load('tutorialdata/embeddings/smallwikicorpus_v2_embeddings.bin')

In [63]:
from gensim.utils import simple_preprocess

class MyDataframeCorpus(object):
    def __init__(self, source_df, text_col, tag_col):
        self.source_df = source_df
        self.text_col = text_col
        self.tag_col = tag_col

    def __iter__(self):
        for i, row in self.source_df.iterrows():
            yield TaggedDocument(words=simple_preprocess(row[self.text_col]), 
                                 tags=[row[self.tag_col]])

corpus_for_doc2vec = MyDataframeCorpus(corpus, 'text', 'category')

https://towardsdatascience.com/implementing-multi-class-text-classification-with-doc2vec-df7c3812824d

In [67]:
from gensim.utils import simple_preprocess

text = []
for i, row in corpus.iterrows():
    text.append(TaggedDocument(words=simple_preprocess(row["text"]), tags=[row["category"]]))

In [71]:
train_documents = text[:5000]
test_documents = text[5000:]

In [72]:
model = Doc2Vec(dm=1, vector_size=300, 
                negative=5, hs=0, min_count=2, sample = 0, 
                workers=6, alpha=0.025, min_alpha=0.001)

In [75]:
model.build_vocab([x for x in train_documents])

In [76]:
from sklearn import utils
train_documents  = utils.shuffle(train_documents)

In [77]:
model.train(train_documents, total_examples=len(train_documents), epochs=30)

In [78]:
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

In [81]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y_train, X_train = vector_for_learning(model, train_documents)
y_test, X_test = vector_for_learning(model, test_documents)

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy for movie plots%s' % accuracy_score(y_test, y_pred))
print('Testing F1 score for movie plots: {}'.format(f1_score(y_test, y_pred, average='weighted')))

/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Testing accuracy for movie plots0.03398558187435633
Testing F1 score for movie plots: 0.034716454602837114


/home/jan/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jan/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [84]:
from sklearn.preprocessing import LabelEncoder



y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)


logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy for movie plots%s' % accuracy_score(y_test, y_pred))
print('Testing F1 score for movie plots: {}'.format(f1_score(y_test, y_pred, average='weighted')))

/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Testing accuracy for movie plots0.12976313079299692
Testing F1 score for movie plots: 0.14011525020086085


/home/jan/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [70]:
a= []

for i, row in corpus.iterrows():
    a.append(TaggedDocument(words=simple_preprocess(row["text"]), tags=[row["category"]]))

TODO: hier weiter, ganz wichtig!!!!!!! hat funktioniert

In [2]:
from keras.preprocessing.text import text_to_word_sequence
array_length = 20 * 300
embedding_features = pd.DataFrame()
for document in corpus["text":
    # Saving the first 20 words of the document as a sequence
    words = text_to_word_sequence(document)[0:20] 
    
    # Retrieving the vector representation of each word and 
    # appending it to the feature vector 
    feature_vector = []
    for word in words:
        try:
            feature_vector = np.append(feature_vector, 
                                       np.array(embeds[word]))
        except KeyError:
            # In the event that a word is not included in our 
            # dictionary skip that word
            pass    # If the text has less then 20 words, fill remaining vector with
    # zeros
    zeroes_to_add = array_length - len(feature_vector)
    feature_vector = np.append(feature_vector, 
                               np.zeros(zeroes_to_add)
                               ).reshape((1,-1))
    
    # Append the document feature vector to the feature table
    embedding_features = embedding_features.append( 
                                     pd.DataFrame(feature_vector))
print embedding_features.shape

SyntaxError: invalid syntax (<ipython-input-2-8048e0230490>, line 2)

In [5]:
# Korpus laden
import pandas as pd
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col=0)

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import numpy as np


labels = LabelEncoder().fit_transform(corpus["category"])
vector  = TfidfVectorizer().fit_transform(corpus["text"])


X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)

  
# Multinomial Naive Bayes
mnb_classifier = MultinomialNB()
mnb = mnb_classifier.fit(X_train, y_train)

# cross validation des Trainingsdatensatzes
mnb_scores = cross_val_score(mnb_classifier, vector, labels, cv=3)
mnb_mean = np.mean(mnb_scores)

print("Der Mittelwert der cross validation bei der  Klassifizierung " 
      + f" mit Multinomial Naive Bayes ist {str(np.around(mnb_mean, decimals=3))}."
      + "\n")


# F1-score des Testdatensatzes
y_pred = mnb_classifier.predict(X_test)
mnb_f1 = f1_score(y_test, y_pred, average="micro")

print("Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist "
      + f"{str(np.around(mnb_f1, decimals=3))}.")

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.859.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.87.


In [10]:
from keras.preprocessing.text import text_to_word_sequence
# define the document
text = 'The quick brown fox jumped over the lazy dog.'
# tokenize the document
result = text_to_word_sequence(text)
print(result)

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']


In [23]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = [corpus["text"][1][:100], corpus["text"][2][:100], corpus["text"][3][:100]]
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

[[0. 1. 1. 2. 2. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [20]:
[corpus["text"][1][:100], corpus["text"][2][:100]]

['Let It Roll : Songs by George Harrison ist das dritte Kompilationsalbum von George Harrison nach der',
 'Lieder wie Orkane ist das dritte offizielle Best-of-Album der deutschen Rockband Böhse Onkelz . Es e']

In [24]:
from keras.preprocessing.text import Tokenizer

train_docs = [corpus["text"][1][:100], corpus["text"][2][:100], corpus["text"][3][:100]]
test_docs = [corpus["text"][4][:100], corpus["text"][5][:100]]

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_docs)

X_train = tokenizer.texts_to_sequences(train_docs)
X_test = tokenizer.texts_to_sequences(test_docs)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(train_docs[2])
print(X_train[2])

Long Stick Goes Boom : The Anthology ist eine Best-of-Kompilation der schweizerischen Hard-Rock-Band
[28, 29, 30, 31, 32, 33, 1, 34, 7, 8, 35, 2, 36, 37, 38, 39]


In [28]:
from nltk import sent_tokenize

corpus = corpus.join(corpus.text.apply(sent_tokenize).rename('sentences'))

In [34]:
l = [text for text in corpus["sentences"]]
train = l[:4800]
test = l[4800:]

In [35]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train)

X_train = tokenizer.texts_to_sequences(train)
X_test = tokenizer.texts_to_sequences(test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(train[2])
print(X_train[2])

['Lieder wie Orkane ist das dritte offizielle Best-of-Album der deutschen Rockband Böhse Onkelz .', 'Es erschien am 2 .', 'Dezember 2011 über das bandeigene Label Rule23 Recordings .', 'Die Kompilation enthält vier CDs und eine DVD mit insgesamt 50 Titeln.Das Album enthält Titel von vielen verschiedenen Alben der Band .', 'Darunter auch Live-Aufnahmen der Lieder Erinnerungen aus Berlin und Nichts ist für die Ewigkeit aus Dortmund .', 'Außerdem sind vier zuvor unveröffentlichte Live-Mitschnitte der Songs Ich bin in dir , Auf gute Freunde , Ein langer Weg und Der Platz neben mir I & II von einem Konzert der Gruppe auf dem Loreley-Felsen im Jahr 2003 enthalten .', 'Diese sind auch visuell auf der DVD festgehalten.Das Albumcover ist schwarz-weiß gehalten .', 'Es zeigt einen gemalten schwarzen Tornado ( Orkan ) , auf dessen Spitze die Gesichter der vier Bandmitglieder , jeweils mit Sonnenbrille , zu sehen sind - von links nach rechts : Stephan Weidner , Kevin Russell , Matthias Röhr und Pet

In [37]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

[1191 1192 1193 1194 1195  262 1196 1197 1198 1199 1200 1201 1202 1203
 1204 1205 1206 1207 1208 1209 1210 1211 1212 1213  438 1214 1215 1216
 1217 1218 1219 1220  438 1221 1222 1223 1224 1225 1226 1227 1228 1229
 1230 1231 1232    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [16]:
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'done': 1, 'well': 1, 'good': 1, 'work': 2, 'great': 1, 'effort': 1, 'nice': 1, 'excellent': 1})


In [7]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train2 = tokenizer.texts_to_sequences(X_train)
X_test2 = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(X_train[2])
print(X_train2[2])

AttributeError: lower not found

- STRG+F "Embedding" --> auch für DL: https://machinelearningmastery.com/start-here/

ICH: hier auch weitere text cleaning dinger

https://towardsdatascience.com/effectively-pre-processing-the-text-data-part-1-text-cleaning-9ecae119cb3e

Wie Boxplots interpretiert werden, kann <a href="https://de.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/box-whisker-plots/a/box-plot-review">hier</a> nachgelesen werden.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()
plt.figure(figsize=(12, 6))
smaller_corpus = corpus.loc[corpus["category"] == "Album nach Typ"]
print(sns.boxplot(y="category", x="length", data=corpus).set(title="Länge der Texte in Zeichen", 
                                                       xlabel='Anzahl der Zeichen', ylabel=''))

NameError: name 'sns' is not defined

https://blog.codecentric.de/2019/03/natural-language-processing-basics/

In [85]:
import logging
import random

import numpy as np
import pandas as pd

from gensim.models import doc2vec, FastText
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


def read_dataset(path):
    dataset = pd.read_csv(path)
    x_train, x_test, y_train, y_test = train_test_split(dataset.text, dataset.category, random_state=0, test_size=0.1)
    x_train = label_sentences(x_train, 'Train')
    x_test = label_sentences(x_test, 'Test')
    all_data = x_train + x_test
    return x_train, x_test, y_train, y_test, all_data


def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the LabeledSentence method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the review.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.LabeledSentence(v.split(), [label]))
    return labeled


def get_vectors(doc2vec_model, corpus_size, vectors_size, vectors_type):
    """Get vectors from trained doc2vec model.
    
        Args:
            doc2vec_model: Trained Doc2Vec model.
            corpus_size: Size of the data.
            vectors_size: Size of the embedding vectors.
            vectors_type: Training or Testing vectors.
        Returns:
            List of vectors.
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = doc2vec_model.docvecs[prefix]
    return vectors


def train_doc2vec(corpus):
    print("Building Doc2Vec vocabulary")
    d2v = doc2vec.Doc2Vec(min_count=1,  # Ignores all words with total frequency lower than this
                          window=15,  # The maximum distance between the current and predicted word within a sentence
                          vector_size=300,  # Dimensionality of the generated feature vectors
                          workers=5,  # Number of worker threads to train the model
                          alpha=0.025,  # The initial learning rate
                          min_alpha=0.00025,  # Learning rate will linearly drop to min_alpha as training progresses
                          dm=1)  # dm defines the training algorithm. If dm=1 means 'distributed memory' (PV-DM)
                                 # and dm =0 means 'distributed bag of words' (PV-DBOW)
    d2v.build_vocab(corpus)

    print("Training Doc2Vec model")
    # 10 epochs take around 10 minutes on my machine (i7), if you have more time/computational power make it 20
    for epoch in range(10):
        print('Training iteration #{0}'.format(epoch))
        d2v.train(corpus, total_examples=d2v.corpus_count, epochs=d2v.epochs)
        # shuffle the corpus
        random.shuffle(corpus)
        # decrease the learning rate
        d2v.alpha -= 0.0002
        # fix the learning rate, no decay
        d2v.min_alpha = d2v.alpha

    logging.info("Saving trained Doc2Vec model")
    d2v.save("d2v.model")
    return d2v

def train_fasttext(corpus):
    fasttext = FastText(size=100, window=5, min_count=5)#, workers=6, sg=1)
        
    fasttext.build_vocab(corpus_file=corpus)
    print("Training Fasttext model")
    
    for epoch in range(10):
        print('Training iteration #{0}'.format(epoch))
        fasttext.train(corpus_file=corpus, total_examples=len(corpus), epochs=fasttext.epochs)#total_examples=fasttext.corpus_count, epochs=fasttext.epochs)
        # shuffle the corpus
        random.shuffle(corpus)
        # decrease the learning rate
        fasttext.alpha -= 0.0002
        # fix the learning rate, no decay
        fasttext.min_alpha = fasttext.alpha
        
    logging.info("Saving trained FastText model")
    fasttext.save("fasttext.model")
    return fasttext


def train_classifier(d2v, training_vectors, training_labels, scores_dict, classification_method):
    logging.info("Classifier training")
    train_vectors = get_vectors(d2v, len(training_vectors), 300, 'Train')
    
    
    if classification_method == "svm":
        model = SVC(gamma="auto", C=2.0, coef0=0.0, kernel="linear")
        model.fit(train_vectors, np.array(training_labels))
        training_predictions = model.predict(train_vectors)
    elif classification_method == "logistic_regression":
        model = LogisticRegression()
        model.fit(train_vectors, np.array(training_labels))
        training_predictions = model.predict(train_vectors)    
    
    
    logging.info('Training predicted classes: {}'.format(np.unique(training_predictions)))
    logging.info('Training accuracy: {}'.format(accuracy_score(training_labels, training_predictions)))
    scores_dict['Training accuracy'] = accuracy_score(training_labels, training_predictions)
    logging.info('Training F1 score: {}'.format(f1_score(training_labels, training_predictions, average='weighted')))
    scores_dict['Training F1 score'] = f1_score(training_labels, training_predictions, average='weighted')
    
    return model, scores_dict


def test_classifier(d2v, classifier, testing_vectors, testing_labels, scores_dict):
    logging.info("Classifier testing")
    test_vectors = get_vectors(d2v, len(testing_vectors), 300, 'Test')
    testing_predictions = classifier.predict(test_vectors)
    logging.info('Testing predicted classes: {}'.format(np.unique(testing_predictions)))
    logging.info('Testing accuracy: {}'.format(accuracy_score(testing_labels, testing_predictions)))
    scores_dict['Testing accuracy'] = accuracy_score(testing_labels, testing_predictions)
    logging.info('Testing F1 score: {}'.format(f1_score(testing_labels, testing_predictions, average='weighted')))
    scores_dict['Testing F1 score'] = f1_score(testing_labels, testing_predictions, average='weighted')
    
    return scores_dict
    
    

def doc2vec_classification(path, classification_method):
    """
    scores_dict = {}
    x_train, x_test, y_train, y_test, all_data = read_dataset(path)
    print(all_data[:100])
    d2v_model = train_doc2vec(all_data)
    classifier, scores_dict = train_classifier(d2v_model, x_train, y_train, scores_dict, classification_method)
    scores_dict = test_classifier(d2v_model, classifier, x_test, y_test, scores_dict)
    return scores_dict

    """

    scores_dict = {}
    x_train, x_test, y_train, y_test, all_data = read_dataset(path)
    d2v_model = train_fasttext(all_data)
    classifier, scores_dict = train_classifier(d2v_model, x_train, y_train, scores_dict, classification_method)
    scores_dict = test_classifier(d2v_model, classifier, x_test, y_test, scores_dict)
    return scores_dict

- Alles über embeddings: https://medium.com/@b.terryjack/nlp-everything-about-word-embeddings-9ea21f51ccfe
- relativ einfache elmo embeddings: https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/